# Import Dependencies

In [51]:
import importlib
import os
from pathlib import Path

import numpy as np
import pandas as pd

import config
import module
from tests import test

for p in [config, module, test]:
    try:
        importlib.reload(p)  # reload module
    except NameError:
        pass

# Load DF


In [52]:
# load DFs
df_att = pd.read_excel(config.path_attendance_data)
df_sess = pd.read_excel(config.path_session_data)
df_trainer = pd.read_excel(config.path_trainer_data, sheet_name=config.month)
df_noncoco = pd.read_excel(config.path_noncoco)
df_coco_member = pd.read_excel(config.path_coco_member)
df_erwin_member = pd.read_excel(config.path_erwin_member)

# Clean and Merge DF


In [53]:
# add data source column to df_sess and df noncoco
# drop index column
# sort column alphabetically
df_sess = (
    df_sess.assign(data_source="coco")
    .drop(columns="index", errors="ignore")
    .loc[:, lambda df_: sorted(df_.columns)]
)
df_noncoco = (
    df_noncoco.assign(data_source="noncoco")
    .drop(columns="index", errors="ignore")
    .loc[:, lambda df_: sorted(df_.columns)]
)
# merge df session and df noncoco
df_sess_full = (
    pd.concat([df_sess, df_noncoco], axis=0)
    .reset_index(drop=True)
    .assign(index=lambda df_: df_.index + 1)
)
df_sess_full.shape

(4235, 21)

## Test


In [54]:
test.test_noncoco_online_class_is_online_location(df_noncoco)
test.test_coco_and_noncoco_cols_same(df_sess, df_noncoco)
test.test_class_exist_in_module_class_grouping(df_sess, "class_type_grouped")
test.test_class_exist_in_module_class_grouping(df_noncoco, "class_type_grouped")
test.test_teacher_center_area_position_not_null(
    df_sess_full, ["teacher_area", "teacher_center", "teacher_position"]
)
test.test_no_trainer_is_duplicated(df_trainer, "coco_teacher_name")
test.test_all_coco_student_centers_are_mapped_in_center_order(
    df_coco_member, "student_center"
)
test.test_all_erwin_student_centers_are_mapped_in_center_order(
    df_erwin_member, "center"
)
test.test_class_type_grouped_mapped_in_class_grouping(
    df_sess_full["class_type_grouped"]
)
test.test_duration_not_null(df_sess_full["class_duration"])

# 1: Class Session & util


In [55]:
class_session = (
    df_sess_full
    # ! exclude GOC
    .loc[df_sess_full["class_mode"] != "GOC"]
    .groupby(["class_grouping", "class_mode", "class_type_grouped"])
    .agg(
        total_scheduled_session=("class_type_grouped", "size"),
        # count how many rows have status "Given"
        total_delivered_session=("class_status", lambda c: (c == "Given").sum()),
        # use sum first because to obtain avg, have to divide sum by num of classes
        total_attendance=("class_attendance", "sum"),
    )
    .assign(
        # get the target attendance for each type of class
        max_attendance=lambda df_: df_.index.get_level_values(2).map(module.class_target),
        avg_attendance=lambda df_: (df_["total_attendance"]
                                    .div(df_["total_delivered_session"])
                                    .astype(float)
                                    .round(1)
                                    .replace(0.0, np.nan)
                                    ),
        avg_per_max_attendance=lambda df_: (df_["avg_attendance"]
                                            .div(df_["max_attendance"])
                                            .round(2)
                                            ),
        percentage_delivered=lambda df_: (df_["total_delivered_session"]
                                          .div(df_["total_scheduled_session"])
                                          .astype(float)
                                          .round(2)
                                          ),
    )
    # sort index to place "Other" below
    .sort_index(level=0, ascending=False)
    # rename and get necessary cols only
    .rename(columns=lambda c: c.replace("_", " ").title())
    .rename_axis(["", "Class Mode", "Class Type Grouped"])
    .loc[:,
         ["Total Scheduled Session", "Total Delivered Session", 
          "Percentage Delivered","Max Attendance","Avg Attendance",
          "Avg Per Max Attendance",]
    ]
)

class_session

Total Scheduled Session  \
         Class Mode Class Type Grouped                                      
VIP      Online     Online VPG                                        144   
                    Online One-on-one                                 276   
         Offline    VPG                                                64   
                    One-on-one                                        405   
Standard Online     Online Social Club                                213   
                    Online First Lesson                                43   
                    Online Complementary                              245   
                    Online Community                                   20   
                    Online Advising Session                           135   
         Offline    Social Club                                       395   
                    Member's Party                                     17   
                    First Lesson                                       82   
                    Complementary                                     326   
                    Community                                          65   
                    Chat Hour                                         136   
                    Advising Session                                    5   
Other    Online     Online Proskill First Lesson                        2   
                    Online Proskill                                     6   
                    Online IELTS First Lesson                           7   
                    Online IELTS                                        7   
         Offline    Proskill First Lesson                               2   
                    Proskill                                           14   
                    Other                                             148   
                    Mock Test                                           6   
                    IELTS First Lesson                                  1   
                    IELTS                                              34   

                                                  Total Delivered Session  \
         Class Mode Class Type Grouped                                      
VIP      Online     Online VPG                                        119   
                    Online One-on-one                                 166   
         Offline    VPG                                                48   
                    One-on-one                                        280   
Standard Online     Online Social Club                                209   
                    Online First Lesson                                43   
                    Online Complementary                              241   
                    Online Community                                   19   
                    Online Advising Session                            92   
         Offline    Social Club                                       322   
                    Member's Party                                     16   
                    First Lesson                                       82   
                    Complementary                                     257   
                    Community                                          55   
                    Chat Hour                                         136   
                    Advising Session                                    4   
Other    Online     Online Proskill First Lesson                        2   
                    Online Proskill                                     6   
                    Online IELTS First Lesson                           7   
                    Online IELTS                                        7   
         Offline    Proskill First Lesson                               2   
                    Proskill                                           14   
                    Other                    

## Test


In [56]:
test.test_all_classes_are_included(df_sess_full, class_session)

# 2: ET Utilization


In [92]:
et_excluded = [
    "Ansyahputri Anggita Rizkiarachma",
]

et_util = (
    df_sess_full
    # merge with df trainer to get working days etc
    .merge(
        df_trainer, 
        how="left", 
        left_on="teacher",
        right_on="coco_teacher_name",
        validate="many_to_one"
    )
    # ! exclude et if less than 1 month
    .loc[lambda df_: ~(df_['teacher'].isin(et_excluded))]
    # to calculate class duration for delivered only,
    # create a temp column
    # if given, leave, if not given, 0
    .assign(
        class_duration_delivered=lambda df_: np.where(
            df_["class_status"] == "Given", df_["class_duration"], 0
        )
    )
    .groupby(["teacher_position_y", "teacher_area_x", "teacher"])
    .agg(
        working_days=("teacher_working_days", "mean"),
        total_scheduled_hours=("class_duration", "sum"),
        total_delivered_hours=("class_duration_delivered", "sum"),
    )
    # filter only for ET and coach
    .loc[
        lambda df_: df_.index.get_level_values("teacher_position_y").isin(["ET", "Coach"])
    ]
    .assign(
        working_days=lambda df_: df_["working_days"].astype("Int64"),
        total_scheduled_hours=lambda df_: df_["total_scheduled_hours"].astype(int),
        total_delivered_hours=lambda df_: df_["total_delivered_hours"].astype(int),
        scheduled_hours_per_day=lambda df_: (
            df_["total_scheduled_hours"]
            .div(df_["working_days"])
            .round(1)
        ),
        delivered_hours_per_day=lambda df_: (
            df_["total_delivered_hours"]
            .div(df_["working_days"])
            .round(1)
        ),
        max_class_hours=lambda df_: module.create_max_hour_per_trainer(df_),
        scheduled_utilization=lambda df_: (
            df_["scheduled_hours_per_day"]
            .div(df_["max_class_hours"])
            .round(2)
        ),
        delivered_utilization=lambda df_: (
            df_["delivered_hours_per_day"]
            .div(df_["max_class_hours"])
            .round(2)
        ),
    )
    .rename(columns=lambda c: c.replace("_", " ").title())
    .rename_axis(["Teacher Position", "Teacher Area", "Teacher"])
)
print(et_util.shape)
et_util.head()

(30, 8)


Working Days  \
Teacher Position Teacher Area Teacher                                    
Coach            BDG          Mustikawati Eka                       16   
                 JKT 1        Hamsah Handayani Ratnasari            14   
                              Mordechai Kaleb Arthur                16   
                              Oktavia Cindy                         18   
                 JKT 2        Hazisyah Alifia Nur                   16   

                                                          Total Scheduled Hours  \
Teacher Position Teacher Area Teacher                                             
Coach            BDG          Mustikawati Eka                                89   
                 JKT 1        Hamsah Handayani Ratnasari                     84   
                              Mordechai Kaleb Arthur                         85   
                              Oktavia Cindy                                  87   
                 JKT 2        Hazisyah Alifia Nur                            91   

                                                          Total Delivered Hours  \
Teacher Position Teacher Area Teacher                                             
Coach            BDG          Mustikawati Eka                                72   
                 JKT 1        Hamsah Handayani Ratnasari                     78   
                              Mordechai Kaleb Arthur                         65   
                              Oktavia Cindy                                  68   
                 JKT 2        Hazisyah Alifia Nur                            73   

                                                          Scheduled Hours Per Day  \
Teacher Position Teacher Area Teacher                                               
Coach            BDG          Mustikawati Eka                                 5.6   
                 JKT 1        Hamsah Handayani Ratnasari                      6.0   
                              Mordechai Kaleb Arthur                          5.3   
                              Oktavia Cindy                                   4.8   
                 JKT 2        Hazisyah Alifia Nur                             5.7   

                                                          Delivered Hours Per Day  \
Teacher Position Teacher Area Teacher                                               
Coach            BDG          Mustikawati Eka                                 4.5   
                 JKT 1        Hamsah Handayani Ratnasari                      5.6   
                              Mordechai Kaleb Arthur                          4.1   
                              Oktavia Cindy                                   3.8   
                 JKT 2        Hazisyah Alifia Nur                             4.6   

                                                          Max Class Hours  \
Teacher Position Teacher Area Teacher                                       
Coach            BDG          Mustikawati Eka                         5.0   
                 JKT 1        Hamsah Handayani Ratnasari              5.0   
                              Mordechai Kaleb Arthur                  5.0   
                              Oktavia Cindy                           5.0   
                 JKT 2        Hazisyah Alifia Nur                     5.0   

                                                          Scheduled Utilization  \
Teacher Position Teacher Area Teacher                                             
Coach            BDG          Mustikawati Eka                              1.12   
                 JKT 1        Hamsah Handayani Ratnasari                    1.2   
                              Mordechai Kaleb Arthur                       1.06   
                              Oktavia Cindy                                0.96   
                 JKT 2        Hazisyah Alifia Nur                          1.14   

                                                          Del

# 2: Ooolab


In [58]:
ooolab = (df_sess_full
          .loc[df_sess_full['teacher_center'] == 'Ooolab']
          .groupby(['class_service', 'class_mode', 'class_type_grouped', 'class_type'])
          .size()
          .to_frame()
          .rename(columns={0: 'count'})
          .rename(columns=lambda c: c.replace('_', ' ').title())
          .rename_axis(['Class Service', 'Class Mode', 'Class Type Grouped', 'Class Type'])
          )
ooolab

Count
Class Service Class Mode Class Type Grouped   Class Type                  
Deluxe & Go   Online     Online Complementary Online Complementary      68
                         Online Social Club   Online Social Club        46
VIP           Online     Online VPG           Online Complementary       3
                                              Online English Corner     60

# 4: Community


## Filter for Community Classes


In [59]:
comms = ["Online Community", "Community"]
df_comm = (
    df_sess_full
    # filter for community
    .loc[df_sess_full["class_type_grouped"].isin(comms)]
    .assign(
        # create com name
        community_name=lambda df_: module.create_com_class(
            df_["class_description"]
        ),
        # create com class type
        community_class_type=lambda df_: module.create_com_class_type(
            df_["class_description"]
        ),
    )
)

## Community Report 1


In [60]:
df_comm_report = (
    df_comm
    .groupby(["community_name", "community_class_type", "class_mode"])
    .agg(
        total_scheduled_session=("community_class_type", "count"),
        total_delivered_session=(
            "class_status", lambda i: (i == "Given").sum()),
        total_attendance=("class_attendance", "sum"),
        avg_attendance=("class_attendance", "mean"),
    )
    .assign(
        avg_attendance=lambda df_: df_[
            "avg_attendance"].astype(float).round(1),
        percentage_delivered=lambda df_: df_["total_delivered_session"]
        .div(df_["total_scheduled_session"])
        .round(2),
    )[
        [
            "total_scheduled_session",
            "total_delivered_session",
            "percentage_delivered",
            "total_attendance",
            "avg_attendance",
        ]
    ]
    .rename(columns=lambda c: c.replace("_", " ").title())
    .rename_axis(["Community", "Class Type", "Class Mode"])
)
df_comm_report

Total Scheduled Session  \
Community Class Type Class Mode                            
CRE-8     Meet Up    Offline                           8   
          Showcase   Online                            1   
          Workshop   Offline                           8   
                     Online                            1   
Leap      Meet Up    Offline                           9   
                     Online                            6   
          Showcase   Online                            1   
          Workshop   Offline                           6   
                     Online                            2   
Re-Charge Meet Up    Offline                          10   
                     Online                            3   
Syndicate Meet Up    Offline                           7   
                     Online                            4   
          Showcase   Offline                           1   
          Workshop   Offline                           7   
                     Online                            1   
UNNAMED   Other      Offline                           9   
                     Online                            1   

                                 Total Delivered Session  \
Community Class Type Class Mode                            
CRE-8     Meet Up    Offline                           8   
          Showcase   Online                            1   
          Workshop   Offline                           8   
                     Online                            1   
Leap      Meet Up    Offline                           9   
                     Online                            6   
          Showcase   Online                            1   
          Workshop   Offline                           6   
                     Online                            2   
Re-Charge Meet Up    Offline                          10   
                     Online                            3   
Syndicate Meet Up    Offline                           7   
                     Online                            4   
          Showcase   Offline                           1   
          Workshop   Offline                           6   
                     Online                            1   
UNNAMED   Other      Offline                           0   
                     Online                            0   

                                 Percentage Delivered  Total Attendance  \
Community Class Type Class Mode                                           
CRE-8     Meet Up    Offline                     1.00              47.0   
          Showcase   Online                      1.00              18.0   
          Workshop   Offline                     1.00              25.0   
                     Online                      1.00               7.0   
Leap      Meet Up    Offline                     1.00              44.0   
                     Online                      1.00              47.0   
          Showcase   Online                      1.00              19.0   
          Workshop   Offline                     1.00              21.0   
                     Online                      1.00              12.0   
Re-Charge Meet Up    Offline                     1.00              40.0   
                     Online                      1.00              28.0   
Syndicate Meet Up    Offline                     1.00              34.0   
                     Online                      1.00              37.0   
          Showcase   Offline                     1.00              18.0   
          Workshop   Offline                     0.86              15.0   
                     Online                      1.00              11.0   
UNNAMED   Other      Offline                     0.00               0.0   
                     Online                      0.00               0.0   

                                 Avg Attendance  
Community Class Type Class Mode                  
CRE-8     Meet Up    Offline  

## Community Report 2


Get the attendance for community classes only.


In [93]:
df_att_comm = (
    df_att
    # create community class type
    .assign(community_class_type=lambda df_: module.create_comm_class_for_att(df_))
    # filter for community
    .loc[lambda df_: df_["community_class_type"] != "NONE"]
    # filter for attendance
    .loc[lambda df_: df_["student_attendance"] == "Attend"]
)

Group.


In [62]:
def catchstate(df, var_name: str) -> 'pd.DataFrame':
    """
    Helper function that captures intermediate Dataframes mid-chain.
    In the global namespace, make a new variable called var_name and set it to dataframe
    """
    globals()[var_name] = df
    return df


df_comm_report_2 = (
    df_att_comm
    .groupby("student_code")
    .agg(num_class_attended=("student_code", "size"))
    .reset_index()
    .groupby("num_class_attended")
    .agg(num_members_who_join_x_class=("num_class_attended", "size"))
    .pipe(catchstate, "temp")
    .reindex(range(1, temp.index.max()+1))
    .fillna(0)
    .astype(int)
    .reset_index()
    .rename(columns=lambda c: c.replace("_", " ").title())
)
df_comm_report_2

,Num Class Attended,Num Members Who Join X Class
0,1,188
1,2,49
2,3,27
3,4,8
4,5,4
5,6,0
6,7,1


## Test


In [83]:
test.test_all_com_classes_are_included(df_sess_full, df_comm_report)
# test.test_total_att_1_eq_total_att_2(df_comm_report, df_comm_report_2)

# 5: Member Cohort


Member cohort processor is moved to its own file.


# 6: Center Population


Center population is a hard problem because center information in Coco can change.

#### Population based on coco


In [69]:
# center_order = pd.CategoricalDtype(config.center_order, ordered=True)

In [70]:
# df_coco_pop = (df_coco_member
#     .assign(
#         # create activity for the current month
#         is_active = lambda df_: module.is_active(
#             df_, "start_date", "end_date", f"1 {config.month[3:]}"
#         ),
#         # get student center and area
#         student_center = lambda df_: df_["student_center"].fillna("Not Specified").astype(center_order),
#         student_area = lambda df_: df_["student_area"].fillna("Not Specified"),
#         consultant = lambda df_: df_["consultant"].str.title(),
#     )
#     # filter only active student
#     .loc[lambda df_: df_["is_active"] == True]
#     # group
#     .groupby(["student_area", "student_center", "student_membership"], observed=True)
#     .agg(
#         num_student=("student_code", "size")
#     )
#     # pivot
#     .reset_index()
#     .pivot(
#         index=["student_area", "student_center"],
#         columns="student_membership"
#     )
#     .sort_index(axis="rows", level=1)
#     .fillna(0)
#     .droplevel(0, axis="columns")
#     .rename_axis(["Area", "Center"], axis="rows")
#     .rename_axis([""], axis="columns")
# )
# df_coco_pop

In [71]:
# df_coco_pop_activity = (df_coco_member
#     .assign(
#         # create activity for the current month
#         is_active = lambda df_: module.is_active(
#             df_, "start_date", "end_date", f"1 {config.month[3:]}"
#         ),
#         # get student center and area
#         student_center = lambda df_: df_["student_center"].fillna("Not Specified").astype(center_order),
#         student_area = lambda df_: df_["student_area"].fillna("Not Specified"),
#         is_class_active=lambda df_: np.where(
#             df_["student_code"].isin((df_att.loc[df_att["student_attendance"] == "Attend", "student_code"].unique())),
#             "Class Active", "Class Inactive"
#         )
#     )
#     # filter only active student
#     .loc[lambda df_: df_["is_active"] == True]
#     # group
#     .groupby(["student_area", "student_center", "is_class_active"], observed=True)
#     .agg(
#         num_student=("student_code", "size")
#     )
#     # pivot
#     .reset_index()
#     .pivot(
#         index=["student_area", "student_center"],
#         columns="is_class_active"
#     )
#     .sort_index(axis="rows", level=1)
#     .fillna(0)
#     .droplevel(0, axis="columns")
#     .rename_axis(["Area", "Center"], axis="rows")
#     .rename_axis([""], axis="columns")
# )
# df_coco_pop_activity

#### Population based on erwin


In [72]:
# df_erwin_pop = (df_erwin_member
#     # filter only active student and coco student
#     .loc[
#         (df_erwin_member[f"active_{config.month[3:].replace(' ', '_')}"] == True) &
#         ~(df_erwin_member["core_product"].isna())
#     ]
#     # cast center as categorical for ordering
#     .assign(
#         center=lambda df_: df_["center"].astype(center_order)
#     )
#     .groupby(["area","center", "core_product"], observed=True)
#     .agg(
#         num_student=("center", "count")
#     )
#     # pivot
#     .reset_index()
#     .pivot(
#         index=["area", "center"],
#         columns="core_product"
#     )
#     .sort_index(axis="rows", level=1)
#     .fillna(0)
#     .droplevel(0, axis="columns")
#     .rename_axis(["Area", "Center"], axis="rows")
#     .rename_axis([""], axis="columns")
# )
# df_erwin_pop

In [73]:
# TODO: add assertion that no members area, center and membership is na or not specified in config.center_order.sum().sum()

# Data: Class Mapping


In [74]:
df_class_mapping = (
    df_sess_full
    # ! exclude GOC
    .loc[df_sess_full["class_mode"] != "GOC"]
    .assign(
        class_grouping=lambda df_: df_[
            "class_type_grouped"].map(module.class_grouping)
    )
    .groupby(["class_grouping", "class_mode", "class_type_grouped", "class_type"])
    .agg(
        total_scheduled_session=("index", "nunique"),
        # count how many rows have status "Given"
        total_delivered_session=(
            "class_status", lambda c: (c == "Given").sum()),
    )
    .rename_axis(["Class Grouping", "Class Mode", "Class Type Grouped", "Class Type"])
    .rename(columns=lambda c: c.replace("_", " ").title())
)

# Save DF


In [84]:
path = (
    Path.cwd() /
    f"output/Exp Management Report (Source) - {config.month_noncoco}.xlsx"
)
sheets = {
    "Class Session & Utilization": class_session,
    "ET Utilization": et_util,
    "Community Class": df_comm_report,
    "Community Class 2": df_comm_report_2,
    # 'Member Pop Erwin': df_erwin_pop,
    # 'Member Pop Coco': df_coco_pop,
    # 'Member Pop Coco - Activity': df_coco_pop_activity,
    "Ooolab": ooolab.reset_index(),
    "Data - Session": df_sess_full,
    "Data - Class Mapping": df_class_mapping,
    "Data - Community Session": df_comm,
}

if not os.path.exists(path):
    module.save_multiple_dfs(sheets, path)
    print("File saved.")
else:
    print("File already exist.")

File saved.


# Experiment

In [76]:
teachers = [
    "Basuki Imelda",
    "Gereau Jason Jarett",
    "Handayani Khaerunisyah Risma",
    "Phillips Toby",
    "Lee Platel Connor",
    "Pratama Dimas Indra",
    "Roach Alex Scott",
    "Oktavia Cindy",
    "Khalisa Fairuz Putri",
    "Jurado Michael John",
    "Lawrence Moore John",
    "Algar Sinclair Alexander John",
    "Johanson Brian",
]
sheets = {}
for teacher in teachers:
    result = (df_sess_full
        .loc[df_sess_full['teacher'] == teacher, ['class_date', 'class_time', 'class_duration']]
        .drop_duplicates()
        .assign(class_hour= lambda df_: df_['class_time'].astype(str).str[:2])
        .groupby('class_date')
        .agg(
            class_hour= ('class_hour', lambda x: ', '.join(x)),
            total_hours= ('class_duration', 'sum')
        )
        .loc[lambda df_: df_['total_hours'] <= 6]
        .reset_index()
        .assign(
                class_date=lambda df_: df_["class_date"].dt.strftime("%d %b"),
                total_hours=lambda df_: df_["total_hours"].astype(int)
        )
    )
    sheets[teacher] = result

In [77]:
path = (
    Path.cwd() /
    f"output/Low Util Trainer - {config.month_noncoco}.xlsx"
)
if not os.path.exists(path):
    module.save_multiple_dfs(sheets, path)
    print("File saved.")
else:
    print("File already exist.")

File already exist.
